In [82]:
with open("input/quijote.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    
chars = sorted(set(raw_text))
vocab_size = len(chars)
print(''.join(chars))
print("Total number of character:", len(raw_text))


 !"'(),-.01234567:;?ABCDEFGHIJLMNOPQRSTUVWXYZ]abcdefghijlmnopqrstuvxyz¡«»¿ÁÉÍÑÓÚàáéíïñóùúü
Total number of character: 2097953


In [83]:
string_to_int = { c:i for i, c in enumerate(chars) }
int_to_string = { i:c for i, c in enumerate(chars) }
encode = lambda strings : [string_to_int[c] for c in strings]
decode = lambda ints : ''.join([int_to_string[i] for i in ints])

encoded = encode("hello pepe")
print(decode(encoded))

hello pepe


In [84]:
import torch
data = torch.tensor(encode(raw_text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([2097953]) torch.int64
tensor([25, 57,  1, 55, 59, 53, 51, 59, 55, 60, 64, 60,  1, 54, 55, 50, 47, 57,
        53, 60,  1, 50, 60, 59,  1, 36, 66, 55, 56, 60, 65, 51,  1, 50, 51,  1,
        57, 47,  1, 32, 47, 59, 49, 54, 47,  0,  0,  0, 39, 21, 38, 21,  0,  0,
        44, 60,  7,  1, 30, 66, 47, 59,  1, 27, 47, 57, 57, 60,  1, 50, 51,  1,
        21, 59, 50, 63, 47, 50, 47,  7,  1, 51, 64, 49, 63, 55, 48, 47, 59, 60,
         1, 50, 51,  1, 23, 82, 58, 47, 63, 47,  1, 50, 51, 57,  1, 37, 51, 69,
         1, 59, 66, 51, 64, 65, 63, 60,  1, 64, 51, 86, 60, 63,  7,  1, 50, 51,
         0, 57, 60, 64,  1, 62, 66, 51,  1, 63, 51, 64, 55, 50, 51, 59,  1, 51,
        59,  1, 64, 66,  1, 23, 60, 59, 64, 51, 56, 60,  7,  1, 49, 51, 63, 65,
        55, 52, 55, 49, 60,  1, 69,  1, 50, 60, 69,  1, 52, 51,  1, 62, 66, 51,
         7,  1, 54, 47, 48, 55, 51, 59, 50, 60,  1, 67, 55, 64, 65, 60,  1, 61,
        60, 63,  0, 57, 60, 64,  1, 64, 51, 86, 60, 63, 51, 64,  1, 50, 83, 57,
      

In [85]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [86]:
block_size = 8
train_data[:block_size+1]

tensor([25, 57,  1, 55, 59, 53, 51, 59, 55])

In [87]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([25]) the target: 57
when input is tensor([25, 57]) the target: 1
when input is tensor([25, 57,  1]) the target: 55
when input is tensor([25, 57,  1, 55]) the target: 59
when input is tensor([25, 57,  1, 55, 59]) the target: 53
when input is tensor([25, 57,  1, 55, 59, 53]) the target: 51
when input is tensor([25, 57,  1, 55, 59, 53, 51]) the target: 59
when input is tensor([25, 57,  1, 55, 59, 53, 51, 59]) the target: 55


In [88]:
torch.manual_seed(1200)
batch_size = 4
block_size = 8 

def get_batch(split):
   data = train_data if split == "train" else val_data
   ix = torch.randint(len(data) - block_size, (batch_size,)) 
   x = torch.stack([data[i:i+block_size] for i in ix])
   y = torch.stack([data[i+1:i+block_size+1] for i in ix])
   return x, y

xb, yb = get_batch("train")
print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)
print("----------")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[48, 84, 47,  1, 48, 66, 63, 57],
        [ 7,  1, 57, 51,  1, 61, 47, 63],
        [51, 59, 65, 63, 51,  1, 50, 60],
        [51, 59, 53, 47, 86, 51,  1, 47]])
targets:
torch.Size([4, 8])
tensor([[84, 47,  1, 48, 66, 63, 57, 47],
        [ 1, 57, 51,  1, 61, 47, 63, 51],
        [59, 65, 63, 51,  1, 50, 60, 64],
        [59, 53, 47, 86, 51,  1, 47,  1]])
----------
when input is tensor([48]) the target: 84
when input is tensor([48, 84]) the target: 47
when input is tensor([48, 84, 47]) the target: 1
when input is tensor([48, 84, 47,  1]) the target: 48
when input is tensor([48, 84, 47,  1, 48]) the target: 66
when input is tensor([48, 84, 47,  1, 48, 66]) the target: 63
when input is tensor([48, 84, 47,  1, 48, 66, 63]) the target: 57
when input is tensor([48, 84, 47,  1, 48, 66, 63, 57]) the target: 47
when input is tensor([7]) the target: 1
when input is tensor([7, 1]) the target: 57
when input is tensor([ 7,  1, 57]) the target: 51
when input is t

In [89]:
xb

tensor([[48, 84, 47,  1, 48, 66, 63, 57],
        [ 7,  1, 57, 51,  1, 61, 47, 63],
        [51, 59, 65, 63, 51,  1, 50, 60],
        [51, 59, 53, 47, 86, 51,  1, 47]])

In [90]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1200)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)
        
        if targets is None:
            loss = None
        else:
            B, T, C  = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1,:]
            probs = F.softmax(logits, dim=1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim = 1)
        return idx
        
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

idx= torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([32, 91])
tensor(4.9582, grad_fn=<NllLossBackward0>)

ZxhTAÍüÚùàÚÁvo.s»bjRT'6!íDzÓy,tttóIBS ¿g5Zóe0gF». -]3!1É¿LT¡ÚfàLWpRa4EÓhX?óIó¡Ú,7dÚ2!vnIàD?Á«üg;XDY4


In [91]:
# Pytorch Optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
batch_size = 32
for steps in range (10000):
    xb, yb = get_batch('train')
    
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

2.318368434906006


In [93]:
idx= torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))


-Po due avan RRosalisuerión ese do nibrraradies que querer co  cumue lalla cos ado fue muer a bro ha
